Collections.abc iis like a type checker

it checks whether the particular object behaves like list or a dict or a set

unlike static type  checker in typing module which checks before the program runs, collections.abc runs at runtime.it checks the behaviour while our program is running.

Collections.abc is a python module that provides Abstract Base Classes (ABCs) for common container types like lists,sets,dicts etc.

without ABCs you might not know how an object bahves like , does it like list,set or dict.

we would have to test it manually , like for example does that object has __ getitem__,  or __ len__ like that.

with this the python standardize the checks.

In [ ]:
from collections.abc import Sequence

isinstance(mylist,Sequence) # True if Sequence.

Now there are three ways of label as ABCs

1. **Inherit Directly**

In [1]:
from collections.abc import Sequence

class MyList(Sequence):
    def __getitem__(self, index):  # Required
        return range(10)[index]
    def __len__(self):             # Required
        return 10

obj = MyList()
print(isinstance(obj, Sequence))  # ✅ True

True


2. **Register as a virtual subclass**

In [2]:
from collections.abc import Sequence

class MyList2:
    def __getitem__(self, index): return range(10)[index]
    def __len__(self): return 10

Sequence.register(MyList2)  # Manually declare
obj = MyList2()
print(isinstance(obj, Sequence))  # ✅ True

True


3. **Duck Typing**

In [3]:
from collections.abc import Iterable

class MyIter:
  def __iter__(self):
    return iter([1,2,3])

print(isinstance(MyIter(), Iterable))  # ✅ True

True


**Example:**

In [9]:
# abcs_demo.py
# Demonstrates *direct inheritance* from collections.abc classes
# and minimal implementations + tiny tests for each.

from __future__ import annotations
from collections.abc import (
    Container, Hashable, Iterable, Iterator, Reversible, Generator,
    Sized, Callable, Collection, Sequence, MutableSequence, ByteString,
    Set, MutableSet, Mapping, MutableMapping,
    MappingView, ItemsView, KeysView, ValuesView,
    Awaitable, Coroutine, AsyncIterable, AsyncIterator, AsyncGenerator
)
import sys
import asyncio
import threading


print(f"Python version: {sys.version}\n")


# ---------------------------
# Helper: run async anywhere (script or Jupyter)
# ---------------------------

def run_async(coro):
    """
    Run an async coroutine both in plain Python and inside Jupyter/IPython.
    - If no event loop is running: use asyncio.run(coro).
    - If a loop is already running (e.g., Jupyter): run in a fresh loop on a new thread.
    """
    try:
        asyncio.get_running_loop()
    except RuntimeError:
        # No running loop -> normal .py execution
        return asyncio.run(coro)
    else:
        # Already in a running loop (Jupyter etc.): run in a separate thread
        result = {}
        error = {}

        def _runner():
            try:
                result["value"] = asyncio.run(coro)
            except BaseException as e:
                error["exc"] = e

        t = threading.Thread(target=_runner, daemon=True)
        t.start()
        t.join()

        if "exc" in error:
            raise error["exc"]
        return result.get("value")


# ---------------------------
# Basic building blocks
# ---------------------------

class MyContainer(Container):
    def __init__(self, data):
        self.data = set(data)

    def __contains__(self, item):  # required by Container
        return item in self.data


class MyHashable(Hashable):
    def __init__(self, key):
        self.key = key

    def __hash__(self):            # required by Hashable
        return hash(self.key)

    def __eq__(self, other):
        return isinstance(other, MyHashable) and self.key == other.key


class MyIterable(Iterable):
    def __init__(self, data):
        self.data = list(data)

    def __iter__(self):            # required by Iterable
        return iter(self.data)


class MyIterator(Iterator):
    def __init__(self, data):
        self._it = iter(data)

    def __iter__(self):            # required by Iterator
        return self

    def __next__(self):            # required by Iterator
        return next(self._it)


class MyReversible(Reversible):
    def __init__(self, data):
        self.data = list(data)

    def __iter__(self):              # Iterable requirement
        return iter(self.data)

    def __reversed__(self):          # Reversible requirement
        return iter(self.data[::-1])


class MyGenerator(Generator):
    """
    Minimal generator-like object implementing the generator protocol.
    Not a 'def ...: yield' generator, but an object matching the ABC contract.
    """
    def __init__(self, data):
        self._it = iter(data)
        self._closed = False

    # Iterator part
    def __iter__(self):
        return self

    def __next__(self):
        if self._closed:
            raise StopIteration
        return next(self._it)

    # Generator protocol
    def send(self, value):
        # For demo, ignore 'value' and just yield next item
        return self.__next__()

    def throw(self, typ, val=None, tb=None):
        raise typ if val is None else typ(val).with_traceback(tb)

    def close(self):
        self._closed = True


class MySized(Sized):
    def __init__(self, data):
        self.data = list(data)

    def __len__(self):
        return len(self.data)


class MyCallable(Callable):
    def __call__(self, x):
        return f"Called with {x}"


# ---------------------------
# Collection / Sequence family
# ---------------------------

class MyCollection(Collection):  # must act like Sized + Iterable + Container
    def __init__(self, data):
        self._data = list(data)

    # Container
    def __contains__(self, item):
        return item in self._data

    # Iterable
    def __iter__(self):
        return iter(self._data)

    # Sized
    def __len__(self):
        return len(self._data)


class MySequence(Sequence):
    def __init__(self, data):
        self._data = list(data)

    def __getitem__(self, idx):
        return self._data[idx]

    def __len__(self):
        return len(self._data)


class MyMutableSequence(MutableSequence):
    def __init__(self, data=()):
        self._data = list(data)

    def __getitem__(self, i):
        return self._data[i]

    def __setitem__(self, i, v):
        self._data[i] = v

    def __delitem__(self, i):
        del self._data[i]

    def __len__(self):
        return len(self._data)

    def insert(self, i, v):
        self._data.insert(i, v)


# Skip ByteString on 3.12+ to avoid deprecation noise
if sys.version_info < (3, 12):
    class MyByteString(ByteString):
        def __init__(self, b: bytes):
            self._b = bytes(b)  # immutable

        def __getitem__(self, i):
            return self._b[i]

        def __len__(self):
            return len(self._b)
else:
    MyByteString = None


# ---------------------------
# Set family
# ---------------------------

class MySet(Set):
    def __init__(self, data=()):
        # store unique elements
        self._data = []
        for x in data:
            if x not in self._data:
                self._data.append(x)

    def __contains__(self, x):
        return x in self._data

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)


class MyMutableSet(MutableSet):
    def __init__(self, data=()):
        self._data = set(data)

    def __contains__(self, x):
        return x in self._data

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def add(self, x):
        self._data.add(x)

    def discard(self, x):
        self._data.discard(x)


# ---------------------------
# Mapping family
# ---------------------------

class MyMapping(Mapping):
    def __init__(self, d: dict):
        self._d = dict(d)

    def __getitem__(self, k):
        return self._d[k]

    def __iter__(self):
        return iter(self._d)

    def __len__(self):
        return len(self._d)


class MyMutableMapping(MutableMapping):
    def __init__(self, d: dict | None = None):
        self._d = {} if d is None else dict(d)

    def __getitem__(self, k):
        return self._d[k]

    def __setitem__(self, k, v):
        self._d[k] = v

    def __delitem__(self, k):
        del self._d[k]

    def __iter__(self):
        return iter(self._d)

    def __len__(self):
        return len(self._d)


# Mapping views (IMPORTANT: use _mapping so MappingView mixins work)
class MyMappingView(MappingView):
    def __init__(self, mapping: MyMapping | MyMutableMapping):
        self._mapping = mapping

    def __len__(self):
        return len(self._mapping)

    def __repr__(self):
        return f"{self.__class__.__name__}({dict(self._mapping)})"


class MyItemsView(ItemsView):
    def __init__(self, mapping: MyMapping | MyMutableMapping):
        self._mapping = mapping

    def __contains__(self, item):
        k, v = item
        return k in self._mapping and self._mapping[k] == v

    def __iter__(self):
        return iter(self._mapping.items())


class MyKeysView(KeysView):
    def __init__(self, mapping: MyMapping | MyMutableMapping):
        self._mapping = mapping

    def __contains__(self, key):
        return key in self._mapping

    def __iter__(self):
        return iter(self._mapping.keys())


class MyValuesView(ValuesView):
    def __init__(self, mapping: MyMapping | MyMutableMapping):
        self._mapping = mapping

    def __contains__(self, value):
        return value in self._mapping.values()

    def __iter__(self):
        return iter(self._mapping.values())


# ---------------------------
# Awaitable / Coroutine
# ---------------------------

class MyAwaitable(Awaitable):
    def __init__(self, value):
        self.value = value

    def __await__(self):
        # simplest way to make an awaitable: delegate to a tiny real coroutine
        async def _coro():
            return self.value
        return _coro().__await__()


class MyCoroutine(Coroutine):
    """
    Wrap a real Python coroutine object and expose the required
    send/throw/close/__await__ methods.
    """
    def __init__(self, value):
        async def _inner():
            return value
        self._coro = _inner()

    def send(self, val):
        return self._coro.send(val)

    def throw(self, typ, val=None, tb=None):
        if val is None:
            return self._coro.throw(typ)
        return self._coro.throw(typ, val, tb)

    def close(self):
        return self._coro.close()

    def __await__(self):
        return self._coro.__await__()


# ---------------------------
# Async Iterable / Iterator / Generator
# ---------------------------

class MyAsyncIterable(AsyncIterable):
    def __init__(self, data):
        self._data = list(data)

    def __aiter__(self):              # ← regular def (not async def)
        return MyAsyncIterator(self._data)


class MyAsyncIterator(AsyncIterator):
    def __init__(self, data):
        self._it = iter(data)

    def __aiter__(self):
        return self

    async def __anext__(self):
        try:
            return next(self._it)
        except StopIteration:
            raise StopAsyncIteration


class MyAsyncGenerator(AsyncGenerator):
    """
    Wrap a real async generator to satisfy the AsyncGenerator ABC.
    """
    def __init__(self, data):
        async def _agen():
            for x in data:
                yield x
        self._agen = _agen()

    def __aiter__(self):
        return self

    async def __anext__(self):
        return await self._agen.__anext__()

    async def asend(self, value):
        return await self._agen.asend(value)

    async def athrow(self, typ, val=None, tb=None):
        if val is None:
            return await self._agen.athrow(typ)
        return await self._agen.athrow(typ, val, tb)

    async def aclose(self):
        return await self._agen.aclose()


# ---------------------------
# Buffer (Python 3.12+)
# ---------------------------

BufferABC = None
try:
    from collections.abc import Buffer as BufferABC  # 3.12+
except Exception:
    pass

if BufferABC is not None:
    class MyBuffer(BufferABC):
        def __init__(self, data: bytes | bytearray | memoryview):
            self._data = memoryview(bytearray(data))

        # Required by Buffer ABC in 3.12+: __buffer__(self, flags, /)
        def __buffer__(self, flags, /):
            return self._data
else:
    MyBuffer = None


# ============================================================
#                       DEMO / TESTS
# ============================================================

def header(title):
    print("\n" + "="*20 + f" {title} " + "="*20)

header("BASIC ABCs")
c = MyContainer([1,2,3])
print(isinstance(c, Container), 2 in c)

h1, h2 = MyHashable("k"), MyHashable("k")
print(isinstance(h1, Hashable), {h1, h2}, h1 == h2)

itbl = MyIterable([1,2,3])
print(isinstance(itbl, Iterable), list(itbl))

itr = MyIterator([10,20])
print(isinstance(itr, Iterator), next(itr), next(itr))
try:
    next(itr)
except StopIteration:
    print("Iterator exhausted")

rev = MyReversible([1,2,3])
print(isinstance(rev, Reversible), list(reversed(rev)))

gen = MyGenerator([7,8,9])
print(isinstance(gen, Generator), next(gen), gen.send(None))
gen.close()
try:
    next(gen)
except StopIteration:
    print("Generator closed")

sz = MySized([1,2,3,4])
print(isinstance(sz, Sized), len(sz))

call = MyCallable()
print(isinstance(call, Callable), call("hello"))

header("COLLECTION / SEQUENCE")
coll = MyCollection([1,2,3])
print(isinstance(coll, Collection), len(coll), 2 in coll, list(coll))

seq = MySequence(["a","b","c"])
print(isinstance(seq, Sequence), seq[1], len(seq), list(reversed(seq)), seq.index("b"))

mseq = MyMutableSequence([1,2,3])
print(isinstance(mseq, MutableSequence), list(mseq))
mseq.append(4); mseq[0] = 10; del mseq[1]; mseq.insert(1, 99)
print("MutableSequence after ops:", list(mseq))

if MyByteString is not None:
    bs = MyByteString(b"hi")
    print(isinstance(bs, ByteString), bs[0], len(bs))
else:
    print("ByteString demo skipped on Python 3.12+ (deprecated).")

header("SET FAMILY")
s = MySet([1,2,2,3])
t = MySet([3,4])
print(
    isinstance(s, Set), len(s), 2 in s,
    list(s & t), list(s | t), list(s - t)  # show contents, not object addresses
)

ms = MyMutableSet([1,2])
print(isinstance(ms, MutableSet), 2 in ms)
ms.add(3); ms.discard(1)
print("MutableSet after ops:", sorted(ms))

header("MAPPING FAMILY")
m = MyMapping({"a":1,"b":2})
print(isinstance(m, Mapping), list(m.keys()), m["a"])

mm = MyMutableMapping({"x":10})
print(isinstance(mm, MutableMapping), list(mm.items()))
mm["y"] = 20; del mm["x"]
print("MutableMapping after ops:", dict(mm))

mv = MyMappingView(m)
print(isinstance(mv, MappingView), repr(mv), len(mv))

iv = MyItemsView(m)
kv = MyKeysView(m)
vv = MyValuesView(m)
print(isinstance(iv, ItemsView), ("a",1) in iv, list(iv))
print(isinstance(kv, KeysView), "b" in kv, list(kv))
print(isinstance(vv, ValuesView), 2 in vv, list(vv))

header("AWAITABLE / COROUTINE / ASYNC")
async def async_section():
    aw = MyAwaitable(123)
    print(isinstance(aw, Awaitable), "Awaitable result:", await aw)

    co = MyCoroutine("ok")
    print(isinstance(co, Coroutine), "Coroutine result:", await co)

    ait = MyAsyncIterator([1,2,3])
    print(isinstance(ait, AsyncIterator), [x async for x in ait])

    aitr = MyAsyncIterable(["x","y"])
    print(isinstance(aitr, AsyncIterable), [x async for x in aitr])

    agen = MyAsyncGenerator([7,8])
    print(isinstance(agen, AsyncGenerator), [x async for x in agen])

# Works in both scripts and notebooks:
run_async(async_section())

header("BUFFER (3.12+)")
if MyBuffer is not None:
    buf = MyBuffer(b"hello")
    print(isinstance(buf, BufferABC), "buffer len:", len(buf.__buffer__(0)))
else:
    print("Buffer ABC not available on this Python (requires 3.12+).")

Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


==================== BASIC ABCs ====================
True True
True {<__main__.MyHashable object at 0x7f07d1e117f0>} True
True [1, 2, 3]
True 10 20
Iterator exhausted
True [3, 2, 1]
True 7 8
Generator closed
True 4
True Called with hello

==================== COLLECTION / SEQUENCE ====================
True 3 True [1, 2, 3]
True b 3 ['c', 'b', 'a'] 1
True [1, 2, 3]
MutableSequence after ops: [10, 99, 3, 4]
ByteString demo skipped on Python 3.12+ (deprecated).

==================== SET FAMILY ====================
True 3 True [3] [1, 2, 3, 4] [1, 2]
True True
MutableSet after ops: [2, 3]

==================== MAPPING FAMILY ====================
True ['a', 'b'] 1
True [('x', 10)]
MutableMapping after ops: {'y': 20}
True MyMappingView({'a': 1, 'b': 2}) 2
True True [('a', 1), ('b', 2)]
True True ['a', 'b']
True True [1, 2]

==================== AWAITABLE / COROUTINE / ASYNC ====================
True Awaitable result: 123
Tr